In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split,Subset
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [10]:
transform = transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize((224,224)),             # resize shortest side to 224 pixels      
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
        ])

In [11]:
AlexNetmodel = models.alexnet(pretrained = True)

In [12]:
for param in AlexNetmodel.parameters():
    param.requires_grad = False

In [13]:
AlexNetmodel.classifier = nn.Sequential(nn.Linear(9216,1024),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.4),
                                        nn.Linear(1024,5),
                                        nn.LogSoftmax(dim=1))

In [14]:
AlexNetmodel

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, 

In [15]:
AlexNetmodel.load_state_dict(torch.load('alex_model2.pth'))

<All keys matched successfully>

In [16]:
import torch
from torchvision import transforms, models
from PIL import Image
import os
import shutil

# Load the saved model
AlexNetmodel.load_state_dict(torch.load('alex_model2.pth'))
AlexNetmodel.eval()

# Define the folder paths
test_folder = 'testing'  # Replace with the path to your testing folder
classified_folders = {
    0: 'f16',
    1: 'f22',
    2: 'a10',
    3: 'c130',
    4: 'v22'
}

# Create the classified folders if they don't exist
for folder in classified_folders.values():
    os.makedirs(folder, exist_ok=True)

# Function to classify and move images
def classify_and_move_images():
    for img_name in os.listdir(test_folder):
        img_path = os.path.join(test_folder, img_name)
        
        # Check if it's an image file
        if img_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
            image = Image.open(img_path).convert('RGB')
            image = transform(image)
            image = image.unsqueeze(0)  # Add batch dimension

            # Predict the class
            with torch.no_grad():
                outputs = AlexNetmodel(image)
                _, predicted = torch.max(outputs, 1)
                class_idx = predicted.item()
            
            # Debug: Print the class index
            print(f"Predicted class index for {img_name}: {class_idx}")

            # Check if the predicted class index is valid
            if class_idx in classified_folders:
                # Move the image to the corresponding folder
                target_folder = classified_folders[class_idx]
                shutil.move(img_path, os.path.join(target_folder, img_name))
                print(f"Moved {img_name} to {target_folder}")
            else:
                print(f"Warning: {img_name} predicted class index {class_idx} is not valid.")

# Run the classification and moving function
classify_and_move_images()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'testing'

In [17]:
import torch
from torchvision import transforms, models
from PIL import Image, ImageDraw, ImageFont
import cv2
import os
import shutil

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.eval()

# Load the classification model)
AlexNetmodel.eval()

# Define the image transformations for classification
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the folder paths
test_folder = 'testing'  # Replace with the path to your testing folder
output_folder = 'output'  # Folder to save annotated images
os.makedirs(output_folder, exist_ok=True)

# Define the classification labels
class_labels = {
    0: 'a10',
    1: 'c130',
    2: 'f16',
    3: 'f22',
    4: 'v22'
}

# Function to detect and classify aircraft in images
def detect_and_classify_images():
    for img_name in os.listdir(test_folder):
        img_path = os.path.join(test_folder, img_name)
        
        # Check if it's an image file
        if img_name.lower().endswith(('png', 'jpg', 'jpeg', 'jfif', 'gif')):
            # Open image using OpenCV
            img_cv2 = cv2.imread(img_path)
            img_pil = Image.fromarray(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))
            
            # Detect objects in the image
            results = yolo_model(img_cv2)
            
            # Convert results to DataFrame
            results_df = results.pandas().xyxy[0]

            # Process detections
            draw = ImageDraw.Draw(img_pil)
            font = ImageFont.load_default()

            for index, row in results_df.iterrows():
                if row['confidence'] > 0.5:  # Filter out low-confidence detections
                    xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

                    # Crop the detected region for classification
                    cropped_image = img_pil.crop((xmin, ymin, xmax, ymax))
                    cropped_image_tensor = transform(cropped_image).unsqueeze(0)

                    # Classify the cropped region
                    with torch.no_grad():
                        outputs = AlexNetmodel(cropped_image_tensor)
                        _, predicted = torch.max(outputs, 1)
                        class_idx = predicted.item()

                    # Draw the bounding box and label on the original image
                    label = class_labels.get(class_idx, "Unknown")
                    draw.rectangle(((xmin, ymin), (xmax, ymax)), outline="red", width=2)
                    draw.text((xmin, ymin - 10), label, fill="red", font=font)
                    print(f"Detected and classified {label} in {img_name}")

            # Save the annotated image
            annotated_img_path = os.path.join(output_folder, img_name)
            img_pil.save(annotated_img_path)
            print(f"Saved annotated image to {annotated_img_path}")

# Run the detection and classification function
detect_and_classify_images()


Using cache found in C:\Users\dell/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-6-11 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'testing'